<a href="https://colab.research.google.com/github/MichaelKlots/algorithms---theory-and-practice-2022-3-aut/blob/main/process_submitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consts and conversions
Which contests to process

In [ ]:
CONTESTS = ['contest_2_g4g_75']
def contest_to_marks(contest):

  return {'q1': 25, 'q2': 35, 'q3': 40}


In [ ]:
import pandas as pd
import numpy as np

# **Inits**

## imports

In [ ]:
import os
os.chdir('/content/drive/MyDrive/MTA/algorithms - theory and practice, 2022-3 aut')

In [ ]:
import importlib
import sys

if 'utils' in sys.modules:
  del sys.modules['utils']
# import utils
import utils
# utils = importlib.reload(utils)

# importlib.reload(utils)


loaded local utils!


In [ ]:
# !pip install -U gspread 
# from google.colab import auth
# auth.authenticate_user()

# import gspread
# from google.auth import default
# creds, _ = default()

# gc = gspread.authorize(creds)

In [ ]:
must_pop = False
# try:
if '/content/drive/MyDrive/code_utils' not in sys.path:
  sys.path.append('/content/drive/MyDrive/code_utils')  
  # must_pop = True
if 'spsh_utils' in sys.modules:
  del sys.modules['spsh_utils']
# import utils
import spsh_utils
# finally:
#   if must_pop:
#     sys.path.remove('/content/drive/MyDrive/code_utils')

## defs

### contest (name str) to submissions DF

In [ ]:
### contest (name str) to submissions DF

def pre_process_contest(contest):

  subs = [s for s in os.listdir(contest) if os.path.isdir(os.path.join(contest, s))]

  for sub in subs:

    files = sorted(os.listdir(os.path.join(contest, sub)))

    for f in files:

      if '.txt' not in f and f == f.lower():
        continue
      
      new_f = f.lower()
      while '.txt' in new_f:
        new_f = new_f.replace('.txt', "")
      
      print(sub)
      print(f)
      print(new_f)

      os.rename(os.path.join(contest, sub, f), 
                os.path.join(contest, sub, new_f))





def _is_sub_file(f):

  if f[0] == 'q' and \
    ('1' in f or '2' in f or '3' in f) and \
    ('.py' in f or '.java' in f or '.cpp' in f):

    return True

  return False


def _files_to_qst_set(files):

  quests = set()
  for q_str in ['q1','q2','q3']:

    if any((f.startswith(q_str) for f in files)):
      quests = quests.union([q_str])

  return quests

def _qst_set_to_max_score(contest, qst_set):

  marks = contest_to_marks(contest)
  
  return sum(marks[q] for q in qst_set)



def get_contest_submissions_df(contest):
  
  subs = [s for s in os.listdir(contest) if os.path.isdir(os.path.join(contest, s))]

  subs_dict = {}

  for sub in subs:
    
    files = sorted(os.listdir(os.path.join(contest, sub)))

    qst_set = _files_to_qst_set(files)

    got_sub = 1#int(any(_is_sub_file(f) for f in files))
    quests = (",".join(qst_set)).replace('q','')
    max_score = _qst_set_to_max_score(contest, qst_set)
    
    
    heb_name = sub.split('_')[0]
    subs_dict[heb_name] = [got_sub, quests, max_score]


  df = pd.DataFrame.from_dict(subs_dict, orient='index', columns = ['submitted', 'questions', 'max score'])

  # df = pd.DataFrame(data={'name_heb': subs_dict.keys(), 'submitted': subs_dict.values()})

  return df

def add_valid_score_col(ranked_df):
  
  valid_ind = ranked_df['Total'].astype(float).fillna(0) <= ranked_df['max score'].astype(float).fillna(0)

  ranked_df['mark valid'] = ''
  ranked_df.loc[~valid_ind, 'mark valid'] = 'X'

  return ranked_df





### G4G contest to relative rank

In [ ]:
### G4G contest df to relative rank df

def get_contest_ranked_df(g4g_contest_df, df_g4g_id):

  ranked_df = pd.merge(g4g_contest_df, df_g4g_id.fillna(''), how = 'right', left_on='Geek', right_on='G4G username')
  ranked_df = ranked_df[[ 'ID_short',
  'Rank',
  'Last Correct Submission :',
  'Total',
  'ID',
  'email',
  'name_heb',
  'G4G username',
  'G4G password',]]


  ranked_df = ranked_df.rename({'Rank': 'G4G rank'}, axis=1)

  ind_got_ranks = ~ranked_df['G4G rank'].isnull()
  # print(ind_got_ranks.sum())

  rel_ranks_df = _get_rel_ranks(ranked_df.loc[ind_got_ranks])

  for col in rel_ranks_df.columns:
    if col not in ranked_df.columns:
      ranked_df[col] = np.nan

  ranked_df.loc[ind_got_ranks] = rel_ranks_df
  
  # ranked_df.loc[ind_got_ranks] = _get_rel_ranks(ranked_df.loc[ind_got_ranks])

  # ranked_df.fillna('', inplace=True)
  # ranked_df = ranked_df.astype(str)



  # ranked_df['G4G rank'] = ranked_df['G4G rank'].astype(float)
  
  
  # assert ranked_df['G4G rank'].dropna().nunique() == len(ranked_df['G4G rank'].dropna()), "duplicate ranks found!"
  
  # ranked_df['MTA rank'] = ranked_df['G4G rank'].rank().astype(float)
  # ranked_df['MTA rank rel'] = 1-(ranked_df['MTA rank']-ranked_df['MTA rank'].min())/(ranked_df['MTA rank'].max()-ranked_df['MTA rank'].min())

  # ranked_df['MTA rank rel'] = ranked_df['MTA rank rel'].astype(str)
  # ranked_df['G4G rank'] = ranked_df['G4G rank'].fillna('')

  return ranked_df

def _get_rel_ranks(red_ranked_df):

  red_ranked_df = red_ranked_df.copy()

  red_ranked_df['G4G rank'] = red_ranked_df['G4G rank'].astype(float)
  
  assert red_ranked_df['G4G rank'].nunique() == len(red_ranked_df['G4G rank']), "duplicate ranks found!"
  
  red_ranked_df['MTA rank'] = red_ranked_df['G4G rank'].rank()
  red_ranked_df['MTA rank rel'] = 1-(red_ranked_df['MTA rank']-red_ranked_df['MTA rank'].min())/(red_ranked_df['MTA rank'].max()-red_ranked_df['MTA rank'].min())

  # red_ranked_df['MTA rank rel'] = red_ranked_df['MTA rank rel']
  # red_ranked_df['G4G rank'] = red_ranked_df['G4G rank'].fillna('')

  return red_ranked_df



### Single contest submission DF to validation suggestion

In [ ]:
### Single contest submission DF to validation suggestion

def mta_rank_to_val_suggestion(ranked_df, agg_val_df, contest):

  assert isinstance(ranked_df, pd.DataFrame)
  
  assert isinstance(agg_val_df, pd.DataFrame)

  assert isinstance(contest, str)


  for col in ['ID_short',
              'ID', 
              'email', 
              'name_heb', 
              'G4G username', 
              'G4G password', 
              'Total',
              'G4G rank',
              'MTA rank',
              'MTA rank rel' ]:

    assert col in ranked_df

  ind = ranked_df['MTA rank rel'].astype(float) > 0.8

  # ind |= (ranked_df['Total'] >= contest_to_marks(contest)['q3'])

  
  val_df = ranked_df.loc[ind, ['ID', 
            'email', 
            'name_heb', 
            'G4G username', 
            'G4G password', 
            'G4G rank',
            'MTA rank',
            'MTA rank rel',
              ]].copy()

  val_df = ranked_df.copy()

  val_df['contest'] = contest
  val_df['code valid'] = 'U'

  cols = val_df.columns.tolist()
  cols.remove('contest')
  cols.remove('code valid')
  cols = ['contest'] + cols + ['code valid']

  val_df = val_df[cols]

  merge_cols = ['contest', 'ID']

  agg_df = pd.concat([val_df, agg_val_df], ignore_index=True)
  agg_df = agg_df.drop_duplicates(merge_cols, keep = 'last', ignore_index = True)

  return agg_df



def re_rank_after_val(ranked_df, agg_val_df, contest):

  agg_val_df = agg_val_df.loc[(agg_val_df['contest']==contest) & (agg_val_df['code valid'].lower()=='x')]
  



  pass


def incorp_code_validation(spsh):

  agg_val_df = spsh_utils.sheet_to_df(spsh.get_sheet_values('manual verifications')).astype(str)
  agg_val_df = mta_rank_to_val_suggestion(ranked_df, agg_val_df, contest)
  re_rank_after_val

  spsh.set_sheet_values_from_df("manual verifications", agg_val_df.fillna(''))

  


## data

In [ ]:
spsh_link = 'https://docs.google.com/spreadsheets/d/1wQr5d827AEf_I9rUceSJ0LSTdcXz0TFE5GMaER1O4BE/'
spsh_id = os.path.basename(spsh_link.rstrip('/'))
spsh = spsh_utils.ggl_spsh_handler(spsh_id)


In [ ]:
# G4G ID
G4G_ID_fname = utils.get_latest_file('G4G_ID', lambda f: f.endswith('csv') and 'G4G_MTA' in f)
print(G4G_ID_fname)
df_g4g_id = pd.read_csv(G4G_ID_fname, dtype=str)
del df_g4g_id['submision_status']
df_g4g_id

G4G_ID/G4G_MTA_33.csv


,ID,email,name_heb,First Name (eng),Last Name (eng),G4G username,G4G password,ID_short
0,207040254,amitkn@mta.ac.il,עמית קפלנסקי,NaN,NaN,NaN,NaN,0254
1,207423625,gursm@mta.ac.il,גור שמואלביץ,NaN,NaN,NaN,NaN,3625
2,204812309,shirto@mta.ac.il,שיר תורן,NaN,NaN,NaN,NaN,2309
3,208494245,zvinaorco@mta.ac.il,צבי נאור כהן,Zvi Naor,Cohen,zvina0e9t,Zvika123,4245
4,207423906,omercn@mta.ac.il,עומר כהן שור,Omer,Cohen Shor,omeru3z5,sarel1234,3906
...,...,...,...,...,...,...,...,...
66,208393926,talmao@mta.ac.il,טל מאור,Tal,Maor,talmmlcg,12345678,3926
67,327280418,alexandrakl@mta.ac.il,אלכסנדרה קולוברסקי,alexandra,kolovarsky,sashkakolovarsky,12345678,0418
68,207071713,idoyk@mta.ac.il,אדו יקותיאל,Ido,Yekutiel,idoyekutkkip,123456789,1713
69,316133222,idanzi@mta.ac.il,עידן זימיליס,Idan,Zimilis,idan0p3p,123456789,3222


In [ ]:
# # student list
# MTA_student_list_fname = utils.get_student_list_fname()
# print(MTA_student_list_fname)
# df_mta_id = pd.read_csv(MTA_student_list_fname, dtype=str)
# df_mta_id

In [ ]:
# df_g4g_id.dropna()|

In [ ]:
print("DONE INITS")

DONE INITS


# Loop contests (and ex's?)

In [ ]:
# Loop contests (and ex's?)

''' Process each contest '''

for contest in CONTESTS:
  
  print('================')
  print("Processing: ", contest)
  pre_process_contest(contest)

  # data = spsh.get_sheet_values(contest + ' G4G')['values']
  # g4g_contest_df = pd.DataFrame.from_records(data[1:], columns = data[0])

  g4g_contest_df = spsh_utils.sheet_to_df(spsh.get_sheet_values(contest + ' G4G'))

  ranked_df = get_contest_ranked_df(g4g_contest_df, df_g4g_id)

  agg_val_df = spsh_utils.sheet_to_df(spsh.get_sheet_values('manual verifications')).astype(str)
  agg_val_df = mta_rank_to_val_suggestion(ranked_df, agg_val_df, contest)
  spsh.set_sheet_values_from_df("manual verifications", agg_val_df.astype(str).fillna(''))
  

  '''
  insert code authenticity validation logic here

  for the top 20%:
    if verification line present - process it and re-rank
    else - add verification requirement

  '''

  ranked_df = ranked_df[['ID_short', 
                         'ID', 
                         'email', 
                         'name_heb', 
                         'G4G username', 
                         'G4G password', 
                         'Total',
                         'G4G rank',
                         'MTA rank',
                         'MTA rank rel' ]]


  submissions_df = get_contest_submissions_df(contest).fillna(0)

  ranked_df = pd.merge(ranked_df, submissions_df, how = 'outer', left_on='name_heb', right_index=True)
  ranked_df['submitted'] = ranked_df['submitted'].fillna(0)
  
  ranked_df = add_valid_score_col(ranked_df)

  raw_mta_sheet = f"{contest} MTA"
  if raw_mta_sheet not in spsh:
    spsh.add_sheet(raw_mta_sheet)

  spsh.set_sheet_values_from_df(raw_mta_sheet+"", ranked_df.fillna(''))


Processing:  contest_2_g4g_75


In [ ]:
assert False, "COMPLETED"

AssertionError: ignored

# Scratchpad Section

In [ ]:
%%shell
pwd
cd contest_2_g4g_75 
pwd
cd "אדו יקותיאל_25423_assignsubmission_file_" 
pwd
find . -name "*q*"

In [ ]:
submissions_df

In [ ]:
agg_val_df.drop_duplicates(['contest','ID'])

In [ ]:
spsh.get_sheets_names()

In [ ]:
raw_mta_sheet = f"{contest} MTA"
print(raw_mta_sheet)
spsh.get_sheets_names()
raw_mta_sheet in spsh

In [ ]:
# g4g_contest_df.dtypes
# df_g4g_id.dtypes
get_contest_ranked_df(g4g_contest_df, df_g4g_id)

In [ ]:
df_g4g_id

In [ ]:
M = pd.merge(g4g_contest_df, df_g4g_id.dropna(), how = 'inner', left_on='Geek', right_on='G4G username')
M = M[[ 'ID_short',
 'Rank',
 'Last Correct Submission :',
 'Total',
 'ID',
 'email',
 'name_heb',
 'G4G username',
 'G4G password',]]

M=M.rename({'Rank': 'G4G rank'}, axis=1)
M['G4G rank'] = M['G4G rank'].astype(int)
assert M['G4G rank'].nunique() == len(M), "duplicate ranks found!"
M['MTA rank'] = M['G4G rank'].rank().astype(int)
M['MTA rank rel'] = 1-(M['MTA rank']-M['MTA rank'].min())/(M['MTA rank'].max()-M['MTA rank'].min())

# lst = [M.columns.to_list()] + M.to_numpy().tolist()
# spsh.set_sheet_values_rows("qwwwwe", lst)

In [ ]:
M

In [ ]:
M['rr']=M['Rank'].astype(int).rank()
M

In [ ]:
get_contest_submissions_df('contest_2_g4g_75')

In [ ]:
CUR_CONTEST = CONTESTS[0]
!echo $CUR_CONTEST > CUR_CONTEST
!cat CUR_CONTEST

In [ ]:
!ls 

In [ ]:
os.listdir('contest_2_g4g_75/נועה מלכה מנצור_25398_assignsubmission_file_')[0].strip()

In [ ]:
!ls -l 'contest_2_g4g_75/נועה מלכה מנצור_25398_assignsubmission_file_'

In [ ]:
%%shell
export CUR_CONTEST=`cat CUR_CONTEST`
find $CUR_CONTEST -type f | grep -vi "zip" | awk  -F"/" '{print $3," --- ", $0}' | grep -vi "^q"  | sort

In [ ]:
%%shell
export CUR_CONTEST=`cat CUR_CONTEST`
find $CUR_CONTEST -type f | grep -vi "zip" | awk  -F"/" '{print $3," --- ", $0}' | grep -v "^q"  |grep -v "^Q"| sort

In [ ]:
assert False

In [ ]:
dir(utils)

In [ ]:
vals=utils.get_spsh_values(spsh_id, "'contest_2_g4g_75 G4G'!A1:F10")
# vals.pop('range')

In [ ]:
# https://docs.google.com/spreadsheets/d/1KgzmUGG4rZLqn8XJJqQKVIbFQ1G5Da_3A1ggof07lAo/edit?usp=sharing